# GGG基盤ガーネット再学習

In [1]:
import sys
sys.path.append('/notebooks/MOD2NN')

In [2]:
import time
import sys
import os
import json

import pandas as pd
from Faraday.two_dim.module.lib.layers import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.9.1
Python: 3.9.13 (main, May 23 2022, 22:01:06) 
[GCC 9.4.0]


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
#@title デフォルトのタイトル テキスト
wavelength = 532.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}

# 連続値

In [5]:
def create_model(**kwargs):
    shape = (100, 100)
    theta = -2.105 * np.pi / 180. / 2.
    eta = np.arctan(0.9785 * np.pi/100. / 2.)
    z = kwargs["z"]
    tf.random.set_seed(kwargs["seed"])
    inputs = tf.keras.Input((28, 28))
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization()(x)
    x = IntensityToElectricField(shape)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=1.0, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=2.0, method='expand')(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=1.0, method='expand')(x)
    x = FaradayRotationByStokes(shape)(x)
    x = MNISTDetector(shape)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [6]:
def train(seed, z):
    model_name = "20230206/z0_{:f}_seed_{:d}".format(z, seed)
    cholab_path = "/content/drive/MyDrive/D2NN/"
    checkpoint_path = "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    path = "trained_model/"+ model_name

    model = create_model(seed=seed, z=z)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,  # category: sparse_categorical_crossentropy
                  metrics=["accuracy"]
                  )

    epochs = 50
    early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='loss',
            min_delta=0.0002,
            patience=2,
        )

    # チェックポイントコールバックを作る
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    logdir = os.path.join(cholab_path +"logs", model_name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    result = model.fit(x_train,
                       y_train,
                       batch_size=64,
                       epochs=epochs,
                       validation_data=(x_val, y_val),
                       callbacks=[cp_callback, tensorboard_callback]
                       )

    model.save(path)

    df = pd.DataFrame(result.history)
    df.to_csv(path + "/history.csv")



    with open(path + "/config.json", 'w') as f:
        json.dump(model.get_config(), f, indent=4)

In [7]:
train(1, 0.5e-3)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing (ImageResizi  (None, 100, 100)         0         
 ng)                                                             
                                                                 
 image_binarization (ImageBi  (None, 100, 100)         0         
 narization)                                                     
                                                                 
 intensity_to_electric_field  (None, 2, 100, 100)      0         
  (IntensityToElectricField)                                     
                                                                 
 angular_spectrum (AngularSp  (None, 2, 100, 100)      0         
 ectrum)                                                     

INFO:tensorflow:Assets written to: trained_model/20230206/z0_0.000500_seed_1/assets


INFO:tensorflow:Assets written to: trained_model/20230206/z0_0.000500_seed_1/assets


# 離散値

In [8]:
def create_model(**kwargs):
    shape = (100, 100)
    theta = -2.105 * np.pi / 180. / 2.
    eta = np.arctan(0.9785 * np.pi / 100. / 2.)
    z = kwargs["z"]
    tf.random.set_seed(kwargs["seed"])
    inputs = tf.keras.Input((28, 28))
    x = ImageResizing(shape)(inputs)
    x = ImageBinarization()(x)
    x = IntensityToElectricField(shape)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=1.0, method='expand')(x)
    x = BinarizedMO(shape, theta=theta, eta=eta, beta=5.)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=2.0, method='expand')(x)
    x = BinarizedMO(shape, theta=theta, eta=eta, beta=5.)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=0.5e-3, d=d, n=1.0, method='expand')(x)
    x = FaradayRotationByStokes(shape)(x)
    x = MNISTDetector(shape)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [9]:
def train(seed, z):
    model_name = "20230206/bi_z0_{:f}_seed_{:d}".format(z, seed)
    cholab_path = "/content/drive/MyDrive/D2NN/"
    checkpoint_path = "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    path = "trained_model/" + model_name

    model = create_model(seed=seed, z=z)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,  # category: sparse_categorical_crossentropy
                  metrics=["accuracy"]
                  )

    epochs = 50
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        min_delta=0.0002,
        patience=2,
    )

    # チェックポイントコールバックを作る
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    logdir = os.path.join(cholab_path + "logs", model_name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    result = model.fit(x_train,
                       y_train,
                       batch_size=64,
                       epochs=epochs,
                       validation_data=(x_val, y_val),
                       callbacks=[cp_callback, tensorboard_callback]
                       )

    model.save(path)

    df = pd.DataFrame(result.history)
    df.to_csv(path + "/history.csv")

    with open(path + "/config.json", 'w') as f:
        json.dump(model.get_config(), f, indent=4)


In [10]:
train(1, 0.5e-3)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 100, 100)         0         
 zing)                                                           
                                                                 
 image_binarization_1 (Image  (None, 100, 100)         0         
 Binarization)                                                   
                                                                 
 intensity_to_electric_field  (None, 2, 100, 100)      0         
 _1 (IntensityToElectricFiel                                     
 d)                                                              
                                                                 
 angular_spectrum_3 (Angular  (None, 2, 100, 100)      0   

INFO:tensorflow:Assets written to: trained_model/20230206/bi_z0_0.000500_seed_1/assets


INFO:tensorflow:Assets written to: trained_model/20230206/bi_z0_0.000500_seed_1/assets
